Name- ROHAN SARASWAT <br>
PRN- 21070126071 <br>
AIML B1

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,RepeatVector,TimeDistributed,Input
from tensorflow.keras.models import Model
from tensorflow.keras.losses import sparse_categorical_crossentropy
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
import re
import string
from string import digits
import numpy as np

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP Lab/Hindi_English_Truncated_Corpus.csv')
#data=pd.read_csv("C:\\Users\\sah-1\\Downloads\\Hindi_English_Truncated_Corpus.csv")
data['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [ ]:
data=data[(data.english_sentence.apply(lambda x:len(str(x))<= 30))&
          (data.hindi_sentence.apply(lambda x:len(str(x))<=30))]

In [ ]:
##changing uppercase to lowercase
data['english_sentence']=data['english_sentence'].apply(lambda x: str(x).lower())
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.lower())

<ipython-input-4-09406c50479e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['english_sentence']=data['english_sentence'].apply(lambda x: str(x).lower())
<ipython-input-4-09406c50479e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: x.lower())


In [ ]:
#Remove quotes
data['english_sentence']=data['english_sentence'].apply(lambda x: re.sub("'",'',x))
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: re.sub("'",'',x))

<ipython-input-5-4c8c98c3bbe8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['english_sentence']=data['english_sentence'].apply(lambda x: re.sub("'",'',x))
<ipython-input-5-4c8c98c3bbe8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: re.sub("'",'',x))


In [ ]:
to_exclude=set(string.punctuation)#set of all special characters
print("Punctuations to exclude:: ",to_exclude)
#remove all the special characters
data['english_sentence']=data['english_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in to_exclude))
data['hindi_sentence']=data['hindi_sentence'].apply(lambda x:''.join(ch for ch in x if ch not in to_exclude))

Punctuations to exclude::  {'~', '.', '`', '<', '@', '!', '#', '\\', '*', ')', ':', '+', ';', '|', '}', '_', '%', ']', "'", '&', '-', '^', '[', '>', '{', '$', '=', '/', '(', '"', '?', ','}


In [ ]:
from string import digits
#Remove all numbers from text
remove_digits=str.maketrans('','',digits)
data['english_sentence']=data['english_sentence'].apply(lambda x:x.translate(remove_digits))

data['hindi_sentence']=data['hindi_sentence'].apply(lambda x:x.translate(remove_digits))

data['hindi_sentence']=data['hindi_sentence'].apply(lambda x:re.sub("[०१२३४५६७८९]","",x))

#Remove Extra Spaces
data['english_sentence']=data['english_sentence'].apply(lambda x:x.strip())

data['hindi_sentence']=data['hindi_sentence'].apply(lambda x:x.strip())

data['english_sentence']=data['english_sentence'].apply(lambda x: re.sub(" +"," ",x))

data['hindi_sentence']=data['hindi_sentence'].apply(lambda x: re.sub(" +"," ",x))

In [ ]:
data.head()

,source,english_sentence,hindi_sentence
11,indic2012,category religious text,श्रेणीधर्मग्रन्थ
23,ted,this changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced,उत्पन्न नहीं कि जाती थी
33,indic2012,maine,मेन
35,ted,can you imagine saying that,क्या आप ये कल्पना कर सकते है


In [ ]:
input_text=[]
target_text=[]
input_characters=set()
target_characters=set()
for eng,hin in data[['english_sentence','hindi_sentence']].itertuples(index=False):
  target='START_ '+hin+' _END' #end sequence
  input_text.append(eng)
  target_text.append(target)
  for eng_char in eng.split():
    if eng_char not in input_characters:
      input_characters.add(eng_char)
  for hin_char in hin.split():
    if hin_char not in target_characters:
      target_characters.add(hin_char)

In [ ]:
print(len(input_text))
print(len(target_text))
print(len(input_characters))
print(len(target_characters))

18416
18416
9232
8665


In [ ]:
print("Input Text ->>>>>"+input_text[0] +"->>>>>>> Output Text ->>>>>>>"+target_text[0])

Input Text ->>>>>category religious text->>>>>>> Output Text ->>>>>>>START_ श्रेणीधर्मग्रन्थ _END


In [ ]:
input_char=sorted(list(input_characters))
target_char=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)+1

max_encoder_seq_length=max([len(txt) for txt in input_text])
max_decoder_seq_length=max([len(txt) for txt in target_text])

In [ ]:
print('Number of samples:',len(input_text))
print('Number of Unique input tokens:',num_encoder_tokens)
print('Number of Unique output tokens:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_length)
print('Max Sequence length for outputs:',max_decoder_seq_length)


Number of samples: 18416
Number of Unique input tokens: 9232
Number of Unique output tokens: 8666
Max Sequence length for inputs: 30
Max Sequence length for outputs: 42


In [ ]:
input_token_index=dict([(word,i+1) for i,word in enumerate(input_char)])
target_token_index=dict([(word,i+1) for i,word in enumerate(target_char)])

In [ ]:
reverse_input_char_index=dict((i,word) for word,i in input_token_index.items())
reverse_target_char_index=dict((i, word) for word, i in target_token_index.items())

In [ ]:
from sklearn.model_selection import train_test_split
X,y=data.english_sentence,data.hindi_sentence
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1,random_state=2)
X_train.shape,X_test.shape

((16574,), (1842,))

In [ ]:
def generate_batch(X,y,batch_size):
  while True:
    for j in range(0,len(X),batch_size):
      encoder_input_data=np.zeros((batch_size,max_encoder_seq_length),dtype='float32')
      decoder_input_data=np.zeros((batch_size,max_decoder_seq_length),dtype='float32')
      decoder_target_data=np.zeros((batch_size,max_decoder_seq_length,num_decoder_tokens),dtype='float32')

      for i,(input_text,target_text) in enumerate(zip(X[j:j+batch_size],y[j:j+batch_size])):
        for t,word in enumerate(input_text.split()):
          encoder_input_data[i,t]=input_token_index[word] #encoder input sequence
        for t,word in enumerate(target_text.split()):
          if t<len(target_text.split())-1:
            decoder_input_data[i,t]=target_token_index[word] #decoder input sequence
          if t>0:
            decoder_target_data[i,t-1,target_token_index[word]]=1.
      yield([encoder_input_data,decoder_input_data],decoder_target_data)


In [ ]:
latent_dim=50


In [ ]:
#Encoder
encoder_inputs=Input(shape=(None,))
enc_emb=Embedding(num_encoder_tokens,latent_dim,mask_zero=True)(encoder_inputs)
encoder_lstm=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder_lstm(enc_emb)

encoder_states=[state_h,state_c]

In [ ]:
#Decoder
decoder_inputs=Input(shape=(None,))
dec_emb_layer=Embedding(num_decoder_tokens,latent_dim,mask_zero=True)
dec_emb=dec_emb_layer(decoder_inputs)

decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_, _=decoder_lstm(dec_emb,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation='softmax')
decoder_outputs=decoder_dense(decoder_outputs)


In [ ]:
model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])


In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 50)             461600    ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 50)             433300    ['input_2[0][0]']             
                                                                                              

In [ ]:
train_samples=len(X_train)
val_samples=len(X_test)
batch_size=16
epochs=100

In [ ]:
model.fit_generator(generator=generate_batch(X_train,y_train,batch_size=batch_size),
                    steps_per_epoch=train_samples//batch_size,epochs=epochs,
                    validation_data=generate_batch(X_test,y_test,batch_size=batch_size),
                    validation_steps=val_samples//batch_size)

Epoch 1/100


<ipython-input-24-455b2635b9a0>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=generate_batch(X_train,y_train,batch_size=batch_size),


1035/1035 [==============================] - 218s 197ms/step - loss: 7.2972 - acc: 0.0476 - val_loss: 6.9997 - val_acc: 0.0611
Epoch 2/100
1035/1035 [==============================] - 189s 182ms/step - loss: 6.5761 - acc: 0.0661 - val_loss: 6.7742 - val_acc: 0.0820
Epoch 3/100
1035/1035 [==============================] - 183s 176ms/step - loss: 6.1172 - acc: 0.0918 - val_loss: 6.6119 - val_acc: 0.1050
Epoch 4/100
1035/1035 [==============================] - 179s 173ms/step - loss: 5.6871 - acc: 0.1226 - val_loss: 6.4706 - val_acc: 0.1260
Epoch 5/100
1035/1035 [==============================] - 184s 178ms/step - loss: 5.3077 - acc: 0.1499 - val_loss: 6.4199 - val_acc: 0.1419
Epoch 6/100
1035/1035 [==============================] - 181s 175ms/step - loss: 4.9699 - acc: 0.1808 - val_loss: 6.3762 - val_acc: 0.1574
Epoch 7/100
1035/1035 [==============================] - 184s 177ms/step - loss: 4.6566 - acc: 0.2142 - val_loss: 6.3548 - val_acc: 0.1681
Epoch 8/100
1035/1035 [================

In [ ]:
model.save_weights('nmt_eng_hin_translation.h5')

Inference

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

In [ ]:
# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [ ]:
dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder␣sequence
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2,initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary
# Final decoder model
decoder_model = Model([decoder_inputs] + decoder_states_inputs,[decoder_outputs2] + decoder_states2)

In [ ]:
def decode_sequence(input_seq):
  # Encode the input as state vectors.
  states_value = encoder_model.predict(input_seq)
  target_seq = np.zeros((1,1))
  # Populate the first character of target sequence with the start character.
  #target_seq[0, 0] = target_token_index['START_ ']
  # Sampling loop for a batch of sequences
  # (to simplify, here we assume a batch of size 1).
  stop_condition = False
  decoded_sentence = ''
  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_target_char_index[sampled_token_index]
    decoded_sentence += ' '+sampled_char
    # Exit condition: either hit max length
    # or find stop character.
    if (sampled_char == ' _END' or len(decoded_sentence) > 25):
      stop_condition = True
    # Update the target sequence (of length 1).
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = sampled_token_index
    # Update states
    states_value = [h, c]
  return decoded_sentence


In [ ]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [ ]:
k+=2
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0])
print('Predicted Hindi Translation:', decoded_sentence[:])

1/1 [==============================] - 0s 26ms/step
Input English sentence: i have a little acorn here
Actual Hindi Translation: मेरे पास एक छोटा सा बाँजफ़ल है
Predicted Hindi Translation:  उल्लेख किया है । हुई थी आप”
